In [1]:
%load_ext tensorboard

from datetime import datetime

import tensorflow as tf
import numpy as np      

#from tqdm import tqdm

In [2]:
import sys, os
from pathlib import Path

In [3]:
sys.path.append('../')

In [4]:
from adinilm.utils import paths_manager as pathsman
from adik.dataloaders.seq2point import MultitargetQuantileRegressionSeq2PointDataLoader

In [5]:
selected_profile = pathsman.PROFILES_DIR / "unetnilm_ukdale_20240321_155419"

print(f"Selected profile {selected_profile.resolve()} {'exists' if selected_profile.exists() else 'does not exist'}")

Selected profile C:\Users\MTinaco\Dev\Solutions\cos-algo-nilm\profiles\unetnilm_ukdale_20240321_155419 exists


In [6]:
denoise_inputs = np.load(selected_profile / "training" / "denoise_inputs.npy")
noise_inputs = np.load(selected_profile / "training" / "noise_inputs.npy")
targets = np.load(selected_profile / "training" / "targets.npy")
states = np.load(selected_profile / "training" / "states.npy")

In [7]:
def split_data(data):
	split_1 = int(0.60 * len(data))
	split_2 = int(0.85 * len(data))
	train = data[:split_1]
	validation = data[split_1:split_2]
	test = data[split_2:]
	return train, validation, test


In [8]:
class CustomDataLoader(tf.keras.utils.Sequence):

        def __init__(self, input, labels, seq_len, indexer, sequence_strategy, batch_size=256):
                self.batch_size = batch_size
                self.backend = sequence_strategy(input, labels, seq_len, indexer)
        def __len__(self):
                return len(self.backend) // self.batch_size

        def __getitem__(self, index):
                batch = [self.backend[idx] for idx in range(index * self.batch_size,(index + 1) * self.batch_size)]
                batch_data, batch_label = zip(*batch)

                batch_data = np.array(batch_data)
                batch_label = np.array(batch_label)

                batch_label = tf.squeeze(batch_label, axis=2)

                return batch_data, tuple([batch_label[:,0,:], batch_label[:,1,:]])


In [9]:
seq_len = 100
denoise = True
dloader = CustomDataLoader(noise_inputs, (states, targets), seq_len=seq_len, indexer=np.arange, sequence_strategy=MultitargetQuantileRegressionSeq2PointDataLoader)


In [10]:
def create_model(input_window_length):

    """Specifies the structure of a seq2point model using Keras' functional API.

    Returns:
    model (tensorflow.keras.Model): The uncompiled seq2point model.

    """
    input_layer = tf.keras.layers.Input(shape=(input_window_length,))
    reshape_layer = tf.keras.layers.Reshape((1, input_window_length, 1))(input_layer)
    conv_layer_1 = tf.keras.layers.Convolution2D(filters=30, kernel_size=(10, 1), strides=(1, 1), padding="same", activation="relu")(reshape_layer)
    conv_layer_2 = tf.keras.layers.Convolution2D(filters=30, kernel_size=(8, 1), strides=(1, 1), padding="same", activation="relu")(conv_layer_1)
    conv_layer_3 = tf.keras.layers.Convolution2D(filters=40, kernel_size=(6, 1), strides=(1, 1), padding="same", activation="relu")(conv_layer_2)
    conv_layer_4 = tf.keras.layers.Convolution2D(filters=50, kernel_size=(5, 1), strides=(1, 1), padding="same", activation="relu")(conv_layer_3)
    conv_layer_5 = tf.keras.layers.Convolution2D(filters=50, kernel_size=(5, 1), strides=(1, 1), padding="same", activation="relu")(conv_layer_4)
    flatten_layer = tf.keras.layers.Flatten()(conv_layer_5)
    label_layer = tf.keras.layers.Dense(1024, activation="relu")(flatten_layer)
    output_layer_1 = tf.keras.layers.Dense(2*5, activation="linear", name="y1_output")(label_layer)
    output_layer_2 = tf.keras.layers.Dense(5*5, activation="linear", name="y2_output")(label_layer)

    model = tf.keras.Model(inputs=input_layer, outputs=[output_layer_1, output_layer_2])
    return model

In [11]:
def create_resnet_model(input_window_length):
    
        input_layer = tf.keras.layers.Input(shape=(input_window_length,))
        reshape_layer = tf.keras.layers.Reshape((1, input_window_length, 1))(input_layer)
        
        conv1 = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 1), strides=(1, 1), padding="same", activation="relu", name='conv1')(reshape_layer)
        bn1 = tf.keras.layers.BatchNormalization(name='bn1')(conv1)
        conv2 = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 1), strides=(1, 1), padding="same", activation="relu", name='conv2')(bn1)
        bn2 = tf.keras.layers.BatchNormalization()(conv2)
        conv3 = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 1), strides=(1, 1), padding="same", activation="relu", name='bn2')(bn2)
        resid_1 = tf.keras.layers.Add()([bn2, conv3])
        relu1 = tf.keras.layers.ReLU(name='relu1')(resid_1)
        bn_res1 = tf.keras.layers.BatchNormalization(name='bn_res1')(relu1)

        conv4 = tf.keras.layers.Conv2D(filters=48, kernel_size=(3, 1), strides=(1, 1), padding="same", activation="relu", name='conv4')(bn_res1)
        bn4 = tf.keras.layers.BatchNormalization(name='bn4')(conv4)
        conv5 = tf.keras.layers.Conv2D(filters=48, kernel_size=(3, 1), strides=(1, 1), padding="same", activation="relu", name='conv5')(bn4)
        bn5 = tf.keras.layers.BatchNormalization(name='bn5')(conv5)
        pool2 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same', name="pool2")(bn5)
        conv6 = tf.keras.layers.Conv2D(filters=48, kernel_size=(3, 1), strides=(1, 1), padding="same", activation="relu", name='conv6')(pool2)
        resid_2 = tf.keras.layers.Add()([pool2, conv6])
        relu2 = tf.keras.layers.ReLU(name="relu2")(resid_2)
        bn_res2 = tf.keras.layers.BatchNormalization(name='bn_res2')(relu2)

        conv7 = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 1), strides=(1, 1), padding="same", activation="relu", name='conv7')(bn_res2)
        bn7 = tf.keras.layers.BatchNormalization(name='bn7')(conv7)
        conv8 = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 1), strides=(1, 1), padding="same", activation="relu", name='conv8')(bn7)
        bn8 = tf.keras.layers.BatchNormalization(name='bn8')(conv8)
        pool3 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same', name='pool3')(bn8)
        conv9 = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 1), strides=(1, 1), padding="same", activation="relu", name='conv9')(pool3)
        resid_3 = tf.keras.layers.Add()([pool3, conv9])
        relu3 = tf.keras.layers.ReLU(name="relu3")(resid_3)
        bn_res3 = tf.keras.layers.BatchNormalization(name="bn_res3")(relu3)

        conv10 = tf.keras.layers.Conv2D(filters=96, kernel_size=(3, 1), strides=(1, 1), padding="same", activation="relu", name="conv10")(bn_res3)
        bn10 = tf.keras.layers.BatchNormalization(name="bn10")(conv10)
        conv11 = tf.keras.layers.Conv2D(filters=96, kernel_size=(3, 1), strides=(1, 1), padding="same", activation="relu", name="conv11")(bn10)
        bn11 = tf.keras.layers.BatchNormalization(name="bn11")(conv11)
        pool4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same', name="pool4")(bn11)
        conv12 = tf.keras.layers.Conv2D(filters=96, kernel_size=(3, 1), strides=(1, 1), padding="same", activation="relu", name="conv12")(pool4)
        resid_4 = tf.keras.layers.Add()([pool4, conv12])
        relu4 = tf.keras.layers.ReLU(name="relu4")(resid_4)
        bn_res4 = tf.keras.layers.BatchNormalization(name="bn_res4")(relu4)

        conv13 = tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 1), strides=(1, 1), padding="same", activation="relu", name="conv13")(bn_res4)
        pool5 = tf.keras.layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='same', name="pool5")(conv13)
        bn13 = tf.keras.layers.BatchNormalization(name="bn13")(pool5)

        conv14 = tf.keras.layers.Conv2D(filters=96, kernel_size=(3, 1), strides=(1, 1), padding="same", activation="relu", name="conv14")(bn13)
        bn14 = tf.keras.layers.BatchNormalization(name="bn14")(conv14)

        conv15 = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 1), strides=(1, 1), padding="same", activation="relu", name="conv15")(bn14)
        bn15 = tf.keras.layers.BatchNormalization(name="bn15")(conv15)

        flatten_layer = tf.keras.layers.Flatten()(bn15)
        
        label_layer = tf.keras.layers.Dense(256, activation="relu", name="dense")(flatten_layer)
        output_layer_1 = tf.keras.layers.Dense(2*5, activation="linear", name="y1_output")(label_layer)
        output_layer_2 = tf.keras.layers.Dense(5*5, activation="linear", name="y2_output")(label_layer)

        model = tf.keras.Model(inputs=input_layer, outputs=[output_layer_1, output_layer_2])
        return model



In [12]:
# mod = create_model(input_window_length=seq_len)
mod = create_resnet_model(input_window_length=seq_len)

dot_img_file = '/tmp/resnet_model.png'
tf.keras.utils.plot_model(mod, to_file=dot_img_file, show_shapes=True)

mod.summary()

You must install graphviz (see instructions at https://graphviz.gitlab.io/download/) for `plot_model` to work.


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 100)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape (Reshape)   │ (None, 1, 100, 1) │          0 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1 (Conv2D)      │ (None, 1, 100,    │        128 │ reshape[0][0]     │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn1                 │ (None, 1, 100,    │        128 │ conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2 (Conv2D)      │ (None, 1, 100,    │      3,104 │ bn1[0][0]         │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 1, 100,    │        128 │ conv2[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn2 (Conv2D)        │ (None, 1, 100,    │      3,104 │ batch_normalizat… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 1, 100,    │          0 │ batch_normalizat… │
│                     │ 32)               │            │ bn2[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ relu1 (ReLU)        │ (None, 1, 100,    │          0 │ add[0][0]         │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_res1             │ (None, 1, 100,    │        128 │ relu1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv4 (Conv2D)      │ (None, 1, 100,    │      4,656 │ bn_res1[0][0]     │
│                     │ 48)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn4                 │ (None, 1, 100,    │        192 │ conv4[0][0]       │
│ (BatchNormalizatio… │ 48)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv5 (Conv2D)      │ (None, 1, 100,    │      6,960 │ bn4[0][0]         │
│                     │ 48)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn5                 │ (None, 1, 100,    │        192 │ conv5[0][0]       │
│ (BatchNormalizatio… │ 48)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool2               │ (None, 1, 50, 48) │          0 │ bn5[0][0]         │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv6 (Conv2D)      │ (None, 1, 50, 48) │      6,960 │ pool2[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 1, 50, 48) │          0 │ pool2[0][0],      │
│                     │                   │            │ conv6[0][0]       │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 353,331 (1.35 MB)

 Trainable params: 351,315 (1.34 MB)

 Non-trainable params: 2,016 (7.88 KB)

In [13]:
input_window_length = 100

In [14]:
class MultiActivationLoss(tf.keras.Loss):
        def __init__(self, *args, **kwargs):
                super().__init__(*args, **kwargs)

                self.states_loss = tf.keras.losses.SparseCategoricalCrossentropy()

        def call(self, y_true, y_pred):

                pred_state = tf.reshape(y_pred, [-1, 5, 2])

                pred_state_softmax = tf.nn.softmax(pred_state, axis=1)

                loss_nll = self.states_loss(y_true, pred_state_softmax)

                return loss_nll

In [15]:
class QuantileLoss(tf.keras.Loss):
        def __init__(self, quantiles=[0.0025,0.1, 0.5, 0.9, 0.975], *args, **kwargs):
                super().__init__(*args, **kwargs)
                self.quantiles = quantiles

        def call(self, y_true, y_pred):
                pred_rms = tf.reshape(y_pred, [-1, 5, 5])

                targets = tf.expand_dims(y_true, axis=1)
                targets = tf.repeat(targets, repeats=[5], axis=1)

                quantiles = tf.convert_to_tensor(self.quantiles, dtype=tf.float32)
                
                error = tf.transpose((targets - pred_rms), perm=[0,2,1])
                loss = tf.math.maximum(quantiles*error, (quantiles-1)*error)
                return tf.reduce_mean(loss, axis=1)

In [16]:
mod.compile(optimizer=tf.keras.optimizers.Adam(),
              loss={'y1_output' : MultiActivationLoss, 'y2_output' : QuantileLoss})

In [17]:
# Define the Keras TensorBoard callback.
# Refer to https://stackoverflow.com/questions/56690089/how-to-graph-tf-keras-model-in-tensorflow-2-0

logdir="../logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
checkpoint_path = f"{logdir}/weights/cp.weights.h5" 
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

mod.fit(dloader, epochs=1, callbacks=[tensorboard_callback, checkpoint_callback])
%tensorboard --logdir logs

c:\Users\MTinaco\Dev\Solutions\cos-algo-nilm\.venv\lib\site-packages\keras\src\callbacks\tensorboard.py:668: UserWarning: Model failed to serialize as JSON. Ignoring... Loss.get_config() missing 1 required positional argument: 'self'
  warnings.warn(f"Model failed to serialize as JSON. Ignoring... {exc}")
c:\Users\MTinaco\Dev\Solutions\cos-algo-nilm\.venv\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


4296/4296 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - loss: 0.6045
Epoch 1: saving model to ../logs/fit/20240731-134858/weights/cp.weights.h5
